In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as opt
import torch.utils.data as tud
import torchvision
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
from torchvision import transforms
transTrain = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=(0.5, 0.5, 0.5),std=(0.5, 0.5, 0.5)),
                                transforms.RandomHorizontalFlip(),transforms.RandomVerticalFlip,transforms.RandomRotation(20) ])
transTest = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=(0.5, 0.5, 0.5),std=(0.5, 0.5, 0.5))])

In [ ]:
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler

batchSize = 50  #the number of samples in a batch for training
valPer = 0.1    #Validation percentage

trainData = datasets.CIFAR10('data/train',download=True,train=True,transform=transTrain)  # Creating Train dataset
testData = datasets.CIFAR10('data/test',download=True,train=False,transform=transTrain)   # Creating Test dataset


trainSize = len(trainData)
idxs = list(range(trainSize))
np.random.shuffle(idxs)
spl = int(np.floor(valPer * trainSize))          #split the dataset to train and validation sets
trainIdxs, valIdxs = idxs[spl:], idxs[:spl]

trainSamp = SubsetRandomSampler(trainIdxs)
valSamp = SubsetRandomSampler(valIdxs)

trainLoader = tud.DataLoader(trainData,batch_size=batchSize,sampler=trainSamp,num_workers=0)
valLoader = tud.DataLoader(trainData,batch_size=batchSize,sampler=valSamp,num_workers=0)
testLoader = tud.DataLoader(testData,batch_size=batchSize,num_workers=0,shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/train/cifar-10-python.tar.gz to data/train


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/test/cifar-10-python.tar.gz to data/test


In [ ]:
#Defining the cnn network
class CNN(nn.Module):
  def __init__(self):
      super(CNN,self).__init__()
      self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
      self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
      self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
      self.fc1 = nn.Linear(64 * 4 * 4, 500)
      self.fc2 = nn.Linear(500, 10)
      self.maxPool = nn.MaxPool2d(kernel_size = 2, stride = 2)
      self.dropout = nn.Dropout(0.25)
      self.relu = nn.ReLU()

  def forward(self,x):
      out = self.maxPool(func.relu(self.conv1(x)))
      out = self.maxPool(func.relu(self.conv2(out)))
      out = self.maxPool(func.relu(self.conv3(out)))
      out = out.view(-1, 64 * 4 * 4)
      out = self.dropout(out)
      out = func.relu(self.fc1(out))
      out = self.dropout(out)
      out = self.fc2(out)
      return out

In [ ]:
#Training the model
import torch.optim as optim

cnnNet = CNN()
cnnNet.to(device)
criterion = nn.CrossEntropyLoss()   # Settin Loss function with criterion
lr = 0.01 #learning rate
optimizer = optim.SGD(cnnNet.parameters(), lr=lr, momentum = 0.9)

nepc = 20 # number of epochs


desiredLoss = 0.001
minLoss = np.Inf

for epc in range(1,nepc+1):
    trainLoss = 0
    cnnNet.train()
    #-----------Train
    for images, labels in trainLoader: 
      images = images.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()
      pred = cnnNet(images)
      loss = criterion(pred, labels)
      loss.backward()
      optimizer.step()
      trainLoss += loss.item()
    trainLoss = trainLoss/len(trainLoader)
    print('Epoch no {} , Loss: {:.4f}'.format(epc, trainLoss))
   
    #-----------Validation
    cnnNet.eval()
    valLoss = 0
    valLoss2 = 0
    for images,labels in valLoader:
      images = images.to(device)
      labels = labels.to(device)
      pred = cnnNet(images)
      loss = criterion(pred, labels)
      valLoss += loss.item()

    valLoss = valLoss/len(valLoader)
    if valLoss < minLoss:  #saving the model with minimum loss
      torch.save(cnnNet.state_dict(), 'cnnNet')
      minLoss = valLoss
      print('Validation loss: {:.4f}'.format(valLoss))
    if valLoss < desiredLoss:
      break


Epoch no 1 , Loss: 1.7826
Validation loss: 1.4285
Epoch no 2 , Loss: 1.3707
Validation loss: 1.2293
Epoch no 3 , Loss: 1.2257
Validation loss: 1.1202
Epoch no 4 , Loss: 1.1158
Validation loss: 1.0362
Epoch no 5 , Loss: 1.0553
Validation loss: 0.9813
Epoch no 6 , Loss: 1.0034
Validation loss: 0.9122
Epoch no 7 , Loss: 0.9631
Epoch no 8 , Loss: 0.9294
Validation loss: 0.9121
Epoch no 9 , Loss: 0.9044
Validation loss: 0.8833
Epoch no 10 , Loss: 0.8783
Validation loss: 0.8581
Epoch no 11 , Loss: 0.8658
Validation loss: 0.8029
Epoch no 12 , Loss: 0.8463
Epoch no 13 , Loss: 0.8333
Validation loss: 0.7967
Epoch no 14 , Loss: 0.8204
Validation loss: 0.7876
Epoch no 15 , Loss: 0.8102
Validation loss: 0.7627
Epoch no 16 , Loss: 0.7966
Epoch no 17 , Loss: 0.7856
Epoch no 18 , Loss: 0.7739
Validation loss: 0.7385
Epoch no 19 , Loss: 0.7718
Epoch no 20 , Loss: 0.7673


In [ ]:
cnnNet.load_state_dict(torch.load('cnnNet'))  #Loading the best model, with minimum error

In [ ]:
testLoss = 0
correct = 0
cnnNet.eval()
total = 0
for images,labels in testLoader:
    images = images.to(device)
    labels = labels.to(device)
    pred = cnnNet(images)
    loss = criterion(pred, labels)
    testLoss += loss.item()*images.size(0)
    total += labels.size(0)
    _, pred = torch.max(pred, 1)
    correct += (pred == labels).sum().item()
testLoss = testLoss/len(testLoader)
print(correct/total*100)
